In [1]:
import os
import openai
import csv
from extraction_package import extraction_functions
from settings import API_KEY, LIBRARY_ID, LIBRARY_TYPE, ZOLTERO_KEY 


client = openai.OpenAI(api_key = API_KEY)

## Commodities Extraction
This will work as the first pass that goes through each of the files and we want to identify all the commodities in each file. We will use chatGPT to do this and then create a master list.

Notes From Goran
- idea: do two commodities --> get the ones mentioned directly for mineral inventory & then another grand list


In [2]:
first_pass_instructions = """You are a geology expert and you are very good in understanding mining reports. 
You will be given a text from a mining report and a table name. From this mining report, you will need to 
extract all of the commodities that are mentioned and are relevant.
"""

get_commodities_prompt = """Given this document, can you return all commodities that are referenced in the 
Mineral Reserve Estimates or Mineral Resource Estimates. The commodities MUST be in this list 
__COMMODITIES_LIST__. Return the found commodities in this JSON format 
{{'commodities': [commodity_1, commodity_2,....]}}. If there are no commodities found, return 
{{'commodities': []}}.
"""

In [3]:
minmod_commodities = extraction_functions.read_csv_to_dict("./codes/minmod_commodities.csv")
commodities_list = []
for key in minmod_commodities:
    commodities_list.append(key['CommodityinGeoKb'])

In [5]:
def create_assistant_commodities(message_file_id):
    assistant = client.beta.assistants.create(
        name="Get Extraction",
        instructions= first_pass_instructions ,
        tools=[{"type": "file_search"}],
        model="gpt-4-1106-preview",
  
    )
    
    thread = client.beta.threads.create(
    messages=[
    {
      "role": "user",
      "content": "You are a geology expert and you are very good in understanding mining reports, which is attached.",
      "attachments": [{ "file_id": message_file_id, "tools": [{"type": "file_search"}] }]
    }]    
        
    )
    
    # print(f"Created an Assistant")
    return thread.id, assistant.id


def check_file_commodities(thread_id, assistant_id, file_path):
    file_instructions = """If the file was correctly uploaded and can be read return YES otherwise return NO. 
                        Only return the Yes or No answer.
                        """
    run = client.beta.threads.runs.create(
      thread_id=thread_id,
      assistant_id=assistant_id,
      instructions= file_instructions
    )
    # print(f"Current run id = {run.id} thread_id = {thread_id}")
    
    extraction_functions.get_completed_assistant_run(thread_id, run.id)
    ans = extraction_functions.get_assistant_message(thread_id, run.id)
    print(f"Response: {ans}")
    if ans.lower() == "no":
        print("We need to reload file.")
        response_code = extraction_functions.delete_assistant(assistant_id)
        if response_code == 200:
            print(f"Deleted assistant {assistant_id}")
        message_file = client.files.create(
              file=open(f"{file_path}", "rb"),
              purpose='assistants'
            )
        new_thread_id, new_assistant_id =  create_assistant_commodities(message_file.id)
        return check_file_commodities(new_thread_id, new_assistant_id, file_path)
    else:
        print("File was correctly uploaded \n")
        return thread_id, assistant_id
    
def add_to_metadata(file_name, commodities_dict):
    csv_file_path = 'commodities_metadata_zinc.csv'

    if not os.path.exists(csv_file_path):
        # Create the CSV file with header row
        with open(csv_file_path, mode='w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['File Name', 'Identified Commodities'])

    with open(csv_file_path, mode='a', newline='') as csvfile:
        # Create a CSV writer object
        writer = csv.writer(csvfile)
        
        if commodities_dict['commodities']:
            joined_commodities = ', '.join(commodities_dict['commodities'])
        else:
            joined_commodities = ""
        
        writer.writerow([file_name, joined_commodities])
    print(f"Finished writing row for {file_name} \n")

In [6]:
directory_path = './reports/zinc/'


file_list = [file_name for file_name in os.listdir(directory_path) if file_name.endswith('.pdf')]

for idx, file_name in enumerate(file_list):
    print(f"Working on File: {file_name} file num: {idx+1} out of {len(file_list)}")
    file_path = directory_path + file_name
    
    file = client.files.create(
    file=open(f"{file_path}", "rb"),
    purpose='assistants'
    )

    thread_id, assistant_id = create_assistant_commodities(file.id)

    thread_id, assistant_id = check_file_commodities(thread_id, assistant_id, file_path)
    run = client.beta.threads.runs.create(
      thread_id=thread_id,
      assistant_id=assistant_id,
      instructions= get_commodities_prompt.replace("__COMMODITIES_LIST__", str(commodities_list))
    )

    extraction_functions.get_completed_assistant_run(thread_id, run.id)
    ans = extraction_functions.get_assistant_message(thread_id, run.id)
    
    correct_format = {'commodities': []}
    commodities_json = extraction_functions.extract_json_strings(ans, str(correct_format), remove_comments = False)
    if commodities_json == None:
        commodities_json = correct_format
        
    print(f"Here are the extracted commodities: {commodities_json}")
    
    extraction_functions.delete_assistant(assistant_id)
    add_to_metadata(file_name, commodities_json)

Working on File: New Polaris Au 2-2019.pdf file num: 1 out of 36
Checking run status: in_progress
Run status: completed

is_failed_result: Response: completed type <class 'str'>
Run: run_ft0A4BI7HlAtv5f969EDci3H Thread: thread_6PVXgvdN5nK2YOHoDrlkAMLZ 
 response: YES 

Response: YES
File was correctly uploaded 

Checking run status: in_progress
Run status: completed

is_failed_result: Response: completed type <class 'str'>
Run: run_IwnQ7AmPG7V7839PJw2PToCM Thread: thread_6PVXgvdN5nK2YOHoDrlkAMLZ 
 response: The only commodity referenced in the attached document that is contained in the provided list is "gold" from the New Polaris Gold Project. It is mentioned in numerous contexts throughout the document, including in reference to drill results, mineral resource estimates, and economic assessments.

Here are some examples from the search results:

- Page 20: "The base case Mineral Resource contains 1.7 Mt of 10.8 g/t Au in the Indicated category; 1.5 Mt of 10.2 g/t Au in the Inferred ca